In [2]:
from py2neo import Graph
from py2neo import Node
from py2neo import Relationship

from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [3]:
graph = Graph(password='xxx')

In [4]:
def get_index_first_False(bool_path):
    found = False
    i = 0
    while not found and i < len(bool_path):
        if bool_path[i] is False:
            found = True
        else:
            i += 1
    return i


def deal_with_revisit(row):
    IDs = row['userIDs']
    boolPath = row['relatedToUkrOrRussia']
    new_boolPath = []
    
    # find how many duplicates
    dup_dict = duplicates(IDs)
    nb_dup = len(dup_dict)
    
    boolPath_ind = 0
    for i, ID in enumerate(IDs):
        if ID not in dup_dict:
            new_boolPath.append(boolPath[boolPath_ind])
            boolPath_ind += 1
        else:
            locs = dup_dict[ID]
            
            if i == locs[0]:
                new_boolPath.append(boolPath[boolPath_ind])
                boolPath_ind += 1
            else:
                new_boolPath.append(boolPath[locs[0]])
                
    return new_boolPath
    
    
def duplicates(seq):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        tally[item].append(i)
    keys = [key for key,loc in tally.items() if len(loc)>1]
    d = dict()
    for key in keys:
        d[key] = tally[key]
    return d  


# Most watched videos

Get the most watched videos that mention Ukraine or Russia. For the authors of these videos, count how many of their videos mention Ukraine or Russia and war. 

In [5]:
query = """
// get the users of the $top most watched videos containing the words "ukraine" or "russia" and "war" 
MATCH (u:user)-[:author]-(v:video)
WHERE (v.desc =~ '(?i).*ukrain.*|.*russia.*' OR v.stickerText =~ '(?i).*ukrain.*|.*russia.*') AND 
(v.desc =~ '(?i).*war.*' OR v.stickerText =~ '(?i).*war.*')
WITH u, v
ORDER BY v.playCount DESC LIMIT $top

// for these users, count how many of their videos mention "ukraine" or "russia" and "war"
MATCH (u)-[:author]-(userVideos:video)
WHERE (userVideos.desc =~ '(?i).*ukrain.*|.*russia.*' OR userVideos.stickerText =~ '(?i).*ukrain.*|.*russia.*') AND 
(userVideos.desc =~ '(?i).*war.*' OR userVideos.stickerText =~ '(?i).*war.*')
WITH u, v, count(userVideos) AS nbVideosMentionUkraineOrRussia

// return info, order by ascending nbVideosMentionUkraineOrRussia and descending nbTimesPlayed
RETURN u.nickname AS name, u.uniqueId AS id, u.signature AS signature, nbVideosMentionUkraineOrRussia, 
u.verified AS verified, u.videoCount AS videos, u.followerCount AS followers, u.followingCount AS isFollowing, 
u.diggCount AS hasLiked, u.heartCount AS hasBeenLiked, u.commerceUserInfo_commerceUser AS cormercial,
v.desc AS videoDesc, v.playCount AS nbTimesPlayed ORDER BY nbVideosMentionUkraineOrRussia, nbTimesPlayed DESC
"""

result_data = graph.run(query, top=100).to_data_frame()
result_data.head()

,name,id,signature,nbVideosMentionUkraineOrRussia,verified,videos,followers,isFollowing,hasLiked,hasBeenLiked,cormercial,videoDesc,nbTimesPlayed
0,stopthwar,stopthwar,🇺🇦🇺🇦🇺🇦,1,0,8,510100,0,251,2300000,NaN,STOP THE WAR 🇺🇦🙏🏻 #ukraine #russia #usa #nato ...,38600000
1,QueenShiels,shielsguardo,NFT: queenshiels\nWebtoon: Love Unreal\nYT: Sh...,1,0,759,106000,1707,45500,3900000,1.0,Connecting the dots and the prologue of #proje...,32100000
2,Cheddy Grace,cheddygrace,Analyst/Part-Time Blogger\nwhatever it is\nBat...,1,0,767,107800,70,5519,14400000,NaN,Emotional moment as Ukraine soldiers left wive...,31600000
3,leok.signs,leo.love.signs,ROAD TO 300K FOLLOWERS!!\nTHANKS FOR ALL SUPPO...,1,0,1158,247000,35,129,19200000,NaN,WAR WORLD 3 IS COMING! 💥#fyp #russia #ukraine ...,20500000
4,hannana👑,hannanajj,RANDOM POST,1,0,475,325500,35,85700,47800000,NaN,I would support you sir #putin #rusia #ukrain...,16900000


Again, get the most watched videos that mention Ukraine or Russia. For the authors of these videos, count how many of their videos mention Ukraine or Russia and war. This time filter out users that mention the word news in their info.

In [6]:
query = """
// get the users of the $top most watched videos containing the words "ukraine" or "russia" and "war" 
// + filter out news accounts
MATCH (u:user)-[:author]-(v:video)
WHERE (v.desc =~ '(?i).*ukrain.*|.*russia.*' OR v.stickerText =~ '(?i).*ukrain.*|.*russia.*') AND 
(v.desc =~ '(?i).*war.*' OR v.stickerText =~ '(?i).*war.*') AND NOT (u.uniqueId =~ '.*[nN]ews.*')
WITH u, v
ORDER BY v.playCount DESC LIMIT $top

// for these users, count how many of their videos mention "ukraine" or "russia" and "war"
MATCH (u)-[:author]-(userVideos:video)
WHERE (userVideos.desc =~ '(?i).*ukrain.*|.*russia.*' OR userVideos.stickerText =~ '(?i).*ukrain.*|.*russia.*') AND 
(userVideos.desc =~ '(?i).*war.*' OR userVideos.stickerText =~ '(?i).*war.*')
WITH u, v, count(userVideos) AS nbVideosMentionUkraineOrRussia

// return info, order by descending nbTimesPlayed
RETURN u.nickname AS name, u.uniqueId AS id, u.signature AS signature, nbVideosMentionUkraineOrRussia, 
u.verified AS verified, u.videoCount AS videos, u.followerCount AS followers, u.followingCount AS isFollowing, 
u.diggCount AS hasLiked, u.heartCount AS hasBeenLiked, u.commerceUserInfo_commerceUser AS cormercial,
v.desc AS videoDesc, v.playCount AS nbTimesPlayed ORDER BY nbTimesPlayed DESC
"""

result_data = graph.run(query, top=100).to_data_frame()
result_data.head()

,name,id,signature,nbVideosMentionUkraineOrRussia,verified,videos,followers,isFollowing,hasLiked,hasBeenLiked,cormercial,videoDesc,nbTimesPlayed
0,Marta Vasyuta,martavasyuta,🇺🇦🇬🇧\nI show real footages of real people\nDon...,4,0,132,263500,159,276800,20100000,NaN,🙏🏽🇺🇦 #Ukraine Spread awareness!,51200000
1,UKRAINE 🇺🇦,foryouukrainee,GOD BLESS UKRAINE 🇺🇦,18,0,51,434300,5,1164,3000000,NaN,#ukraine #viralvideo #army #russia #fypシ,48800000
2,Valerisssh,valerisssh,PHOTOGRAPHER \nHOW TO HELP UKRAINE?\nlink on m...,18,0,626,1100000,87,21100,38200000,NaN,Living my best life 🥰🥰🥰 Thanks Russia! #ukrain...,48100000
3,🔥🔥,nato.1234,,2,0,8,205300,1,82,4600000,1.0,#russia #ukraine #russiavsukraine #war #ww3 #v...,43800000
4,Brut.,brutofficiel,Abonne-toi ↖️\n\n\n\nACTU. DÉCRYPTAGE. INTERVI...,3,1,1533,1600000,45,413,33900000,1.0,« Stop Putin’s War » : action des femen à Pari...,39200000


Get the most played videos.

In [7]:
query = """
MATCH (u:user)-[:author]-(v:video)
WITH u, v
ORDER BY v.playCount DESC LIMIT $top
RETURN u.nickname AS name, u.uniqueId AS id, u.signature AS signature, u.verified AS verified, 
u.videoCount AS videos, u.followerCount AS followers, u.followingCount AS isFollowing, u.diggCount AS hasLiked, 
u.heartCount AS hasBeenLiked, u.commerceUserInfo_commerceUser AS cormercial,v.desc AS videoDesc, 
v.playCount AS nbTimesPlayed ORDER BY nbTimesPlayed DESC
"""

result_data = graph.run(query, top=100).to_data_frame()
result_data.head()

,name,id,signature,verified,videos,followers,isFollowing,hasLiked,hasBeenLiked,cormercial,videoDesc,nbTimesPlayed
0,Bella Poarch,bellapoarch,This ain’t Build a B*tch💅🏻💕,1,463,89400000,590,8348,2100000000,NaN,To the 🐝 🐝 🐝 #fyp,686500000
1,Jamie Big Sorrel Hor,jamie32bsh,Great Things Coming This Week!,0,62,7800000,72,3420,93900000,NaN,,413900000
2,Kapsalonfreedom,kapsalonfreedomboxmeer,No bio yet,0,316,5400000,1,18900,77500000,NaN,#viral #viral#foryou #foryou #foryoupage #kaps...,349900000
3,Patrick Hanson,patrickhanson17,,0,37,4100000,48,2895,39300000,NaN,#trickshot #pingpong #rubegoldberg #quarantine...,325300000
4,Kison Kee,kisonkee,@kisonkee on Instagram \nMGMT: Clicks Talent \...,1,161,3700000,349,40900,69400000,NaN,You guys (literally) asked for this 😂 #comedy ...,319600000


# Recommendations

Get the total number of recommendations.

In [8]:
query = """
MATCH p = ()-[r:recommends]->() RETURN count(p) AS nbRecommendations
"""
result_data = graph.run(query).to_data_frame()
result_data.head()

,nbRecommendations
0,228389


Get the most recommended videos and who they were recommended by.

In [9]:
query = """
// get all videos that have been recommended
MATCH (recommended:video)<-[:recommends]-(recommender)

// count the number of times the video has been recommended, return a list of the user IDs and video descriptions of the
// recommenders, return whether the video was recommended by only users, only videos, or both
WITH recommended, count(recommender) AS nbTimesRecommended, 
collect(coalesce(recommender.desc, recommender.uniqueId)) AS recommenderInfo, 
collect(distinct labels(recommender)[0]) AS  recommenderLabel
RETURN nbTimesRecommended, recommended.playCount AS timesPlayed, recommended.desc AS desc, recommenderLabel, recommenderInfo

// get the top $top recommended videos 
ORDER BY nbTimesRecommended DESC, timesPlayed DESC
LIMIT $top
"""
result_data = graph.run(query, top=100).to_data_frame()
result_data.head()

,nbTimesRecommended,timesPlayed,desc,recommenderLabel,recommenderInfo
0,14,1900000,Too early for easter chocolate?,[video],[#الجزائر🇩🇿😘تيك_توك_ #russia #ukraine #dz #vir...
1,13,24000000,Who else cleans out there exhaust? 😂#foryou #f...,[video],"[analisi top #perte #foryou #foryoupage #fyp, ..."
2,12,6200000,Ready when you are prince!🐣🤍#9monthspregnant #...,[video],"[analisi top #perte #foryou #foryoupage #fyp, ..."
3,12,4900000,Teaching her patience 😂😂😂 @jlo1017 #fyp #Screa...,[video],[Wat zou jij doen als dit jou overkomt?🤣 2✨ #N...
4,12,1300000,Up there with one of the worst messages to rec...,[video],"[analisi top #perte #foryou #foryoupage #fyp, ..."


Get the most recommended users and who they were recommended by.

In [10]:
query = """
// get all users that have been recommended
MATCH (recommended:user)<-[:recommends]-(recommender)

// count the number of times the user has been recommended, return a list of the user IDs and video descriptions of the
// recommenders, return whether the user was recommended by only users, only videos, or both
WITH recommended, count(recommender) AS nbTimesRecommended, 
collect(coalesce(recommender.desc, recommender.uniqueId)) AS recommenderInfo, 
collect(distinct labels(recommender)[0]) AS recommenderLabel
RETURN nbTimesRecommended, recommended.uniqueId AS ID, recommended.signature AS signature, recommended.videoCount AS nbVideos, recommended.followerCount as followers, recommenderLabel, recommenderInfo
ORDER BY nbTimesRecommended DESC, followers DESC
LIMIT $top
"""
result_data = graph.run(query, top=100).to_data_frame()
result_data.head()

,nbTimesRecommended,ID,signature,nbVideos,followers,recommenderLabel,recommenderInfo
0,151,maliciadorable,Insta : Maliciadorable ⤴️🤳\n💌 maliciaetsalva@p...,851.0,3000000.0,[user],"[thememehub.v2, edxraya, 20minutesonline, cush..."
1,149,nicoenvrai,Demandes Pro : nicolas@artistikprod.com\nEn sp...,598.0,1200000.0,[user],"[ukraine_.saveukraine, edxraya, 20minutesonlin..."
2,148,celinedept,Go follow my insta 👆🏼\n📩 cemi@fantube.me\nBelg...,1119.0,12400000.0,[user],"[d.frroku, geybn, mayor_league, navoiteam, aln..."
3,142,badfifii,𝐈𝐍𝐒𝐓𝐀𝐆𝐑𝐀𝐌: @badfifii\n📩 fiona@tayaragency.com,1129.0,1100000.0,[user],"[elifbsk, edxraya, cush_push, stayscrappy, joh..."
4,141,7futsal,Everything Futsal ⚽\nNew YouTube Video\nCLICK ...,2006.0,2200000.0,[user],"[ukraine_.saveukraine, edxraya, cush_push, sta..."


Count how many times a videos were recommended by users and vice versa.

In [11]:
query = """
MATCH p = (u:user)<-[:recommends]-(v:video)
RETURN count(p) AS count
"""
result_data = graph.run(query).to_data_frame()
result_data.head()

,count
0,0


Get how many times videos were recommended by videos.

In [12]:
query = """
MATCH p = (recommended:video)<-[:recommends]-(recommender:video)
RETURN count(p) AS count
"""
result_data = graph.run(query).to_data_frame()
result_data.head()

,count
0,37165


Get how many times users were recommended by users.

In [13]:
query = """
MATCH p = (recommended:user)<-[:recommends]-(recommender:user)
RETURN count(p) AS count
"""
result_data = graph.run(query).to_data_frame()
result_data.head()

,count
0,191224


Get the most recommended videos and see whether the authors were ever recommended.

In [14]:
query = """
// get the most recommended videos
MATCH (recommended:video)<-[:recommends]-()
WITH recommended, count(*) AS nbTimesVidRec
ORDER BY nbTimesVidRec DESC LIMIT $top

// get the authors of these videos
MATCH (u:user)-[:author]->(recommended)
WITH u, recommended.desc AS desc, nbTimesVidRec

// count how many times the authors have been recommended
OPTIONAL MATCH p = (u)<-[:recommends]-()
WITH u, desc, nbTimesVidRec, count(p) AS nbTimesAuthorRec
RETURN nbTimesVidRec, desc, u.uniqueId AS id, u.signature AS signature, u.followerCount AS followers, nbTimesAuthorRec
ORDER BY nbTimesVidRec DESC, nbTimesAuthorRec DESC
"""
result_data = graph.run(query, top=100).to_data_frame()
result_data.head()

,nbTimesVidRec,desc,id,signature,followers,nbTimesAuthorRec
0,14,Too early for easter chocolate?,tikbics,🌟FOLLOW🌟 if you love chocolate 🤤🍫,52800,1
1,13,Who else cleans out there exhaust? 😂#foryou #f...,tobyies,Car life 🚗,36900,0
2,12,Up there with one of the worst messages to rec...,lucinda,✌🏼& 😚,721300,1
3,12,Ready when you are prince!🐣🤍#9monthspregnant #...,hugo_hossam,🇹🇿🇴🇲\nHossam.ebony@outlook.com ✉️\nLabour Stor...,205600,0
4,12,Teaching her patience 😂😂😂 @jlo1017 #fyp #Screa...,mamalindy,"💕 fun, love and laughter 💕\nLindy@select.co",8900000,0


Get the most recommended users and see how many of their videos were recommended (`nbVideosRec`) and how many times a video of their's was recommended (`nbTotalRec`).

In [15]:
query = """
// get the most recommended users
MATCH (recommended:user)<-[:recommends]-()
WITH recommended, count(*) AS nbTimesUserRec
ORDER BY nbTimesUserRec DESC LIMIT $top

// get the videos of these users
MATCH (v:video)<-[:author]-(recommended)
WITH v, recommended, nbTimesUserRec

// count how many of the user's videos have been recommended (size(collect(...))), and how many times they have been 
// recommended in total (count(p))
OPTIONAL MATCH p = (v)<-[:recommends]-()
WITH recommended, nbTimesUserRec, count(p) AS nbTotalRec, 
size(
    collect(DISTINCT
    CASE 
        WHEN p IS NOT null THEN v.id
    END)) AS nbVideosRec
RETURN nbTimesUserRec, recommended.uniqueId AS id, recommended.signature AS signature, 
recommended.followerCount AS followers, nbTotalRec, nbVideosRec
ORDER BY nbTimesUserRec DESC, nbTotalRec DESC
"""
result_data = graph.run(query, top=100).to_data_frame()
result_data.head()

,nbTimesUserRec,id,signature,followers,nbTotalRec,nbVideosRec
0,151,maliciadorable,Insta : Maliciadorable ⤴️🤳\n💌 maliciaetsalva@p...,3000000,10,6
1,149,nicoenvrai,Demandes Pro : nicolas@artistikprod.com\nEn sp...,1200000,44,18
2,148,celinedept,Go follow my insta 👆🏼\n📩 cemi@fantube.me\nBelg...,12400000,14,8
3,142,badfifii,𝐈𝐍𝐒𝐓𝐀𝐆𝐑𝐀𝐌: @badfifii\n📩 fiona@tayaragency.com,1100000,0,0
4,141,7futsal,Everything Futsal ⚽\nNew YouTube Video\nCLICK ...,2200000,16,9


# Weakly connected components

## Whole graph

Create a graph projection for the enitre graph.

In [16]:
query = """
// create a native projection of the whole graph
CALL gds.graph.project($name, '*', '*')
YIELD *
"""
graph.run(query, name='whole_graph')

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{__ALL__: {label: '*', properties: {}}}","{__ALL__: {orientation: 'NATURAL', aggregation: 'DEFAULT', type: '*', properties: {}}}",whole_graph,427275,1133997,2199


Get the 15 largest weakly connected components.

In [17]:
query = """
CALL gds.wcc.stream($name) YIELD nodeId, componentId
RETURN componentId, count(*) AS size
ORDER BY size DESC LIMIT $limit
"""
result_data = graph.run(query, name='whole_graph', limit=15).to_data_frame()
result_data

,componentId,size
0,0,426987
1,10815,57
2,3805,20
3,5787,10
4,976,8
5,22774,3
6,25532,3
7,44853,2
8,44815,2
9,44848,2


## Videos

Create a graph projection for the videos and `recommends` relationship.

In [18]:
query = """
CALL gds.graph.project(
    $name, 
    'video', 
    'recommends'
)
YIELD *
"""
graph.run(query, name='video_recommendations')

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{video: {label: 'video', properties: {}}}","{recommends: {orientation: 'NATURAL', aggregation: 'DEFAULT', type: 'recommends', properties: {}}}",video_recommendations,137319,37165,1124


Get the 15 largest weakly connected components.

In [19]:
query = """
CALL gds.wcc.stream($name) YIELD nodeId, componentId
RETURN componentId, count(*) AS size
ORDER BY size DESC LIMIT $limit
"""
result_data = graph.run(query, name='video_recommendations', limit=15).to_data_frame()
result_data

,componentId,size
0,82664,22800
1,84793,7
2,84960,2
3,83601,2
4,83958,2
5,84939,2
6,82975,2
7,83175,2
8,83336,2
9,83557,2


For each wcc, get the number of times Ukraine or Russia are mentioned in the video description or sticker text, the tags of the video, or the songs. The tags and songs can have duplicates, for example, if 2 videos in a component have the same tag, the tag would be counted 2 times.

In [20]:
query = """
// get the wcc's 
CALL gds.wcc.stream($name) YIELD nodeId, componentId
WITH componentId,  count(*) AS size, collect(gds.util.asNode(nodeId)) AS videos

// count the number of times the videos mention Ukraine or Russia
WITH componentId, size, videos,
size(
    [vid IN videos WHERE (
        vid.desc =~ $regex OR
        vid.stickerText =~ $regex        
    )]
) AS countForVids

// use unwind so the videos can be read as rows and used for matching
UNWIND videos AS v
WITH componentId, size, videos, v, countForVids

// count the tags related to the videos (if there are any) that mention Ukraine or Russia
OPTIONAL MATCH (t:tag)--(v)
WHERE t.title =~ $regex OR t.desc =~ $regex OR t.id =~ $regex OR t.splitTitle =~ $regex
WITH componentId, size, videos, v, countForVids, size(collect(t)) AS countForTags

// count the songs related to the videos (if there are any) that mention Ukraine or Russia
OPTIONAL MATCH (s:song)--(v)
WHERE s.id =~ $regex OR s.title =~ $regex OR s.album =~ $regex
WITH componentId, size, countForVids, countForTags, size(collect(s)) AS countForSongs

// the counts for the tags and songs are split over the different videos in the components (because UNWIND was used 
// to read the list of videos as rows), sum them to get the total count for the component
WITH componentId, size, countForVids, sum(countForTags) AS totalCountForTags, sum(countForSongs) AS totalCountForSongs

RETURN componentId, size, countForVids + totalCountForTags + totalCountForSongs AS totalCount, 
countForVids, totalCountForTags, totalCountForSongs
ORDER BY size DESC, totalCount DESC, countForVids DESC, totalCountForTags DESC, totalCountForSongs DESC LIMIT $limit
"""
result_data = graph.run(query, name='video_recommendations', regex="(?i).*ukrain.*|.*russia.*", limit=100).to_data_frame()
result_data

,componentId,size,totalCount,countForVids,totalCountForTags,totalCountForSongs
0,82664,22800,769,656,97,16
1,84793,7,0,0,0,0
2,83336,2,5,1,4,0
3,85101,2,5,1,4,0
4,83557,2,5,1,4,0
...,...,...,...,...,...,...
95,25640,1,10,1,9,0
96,66615,1,10,1,9,0
97,85264,1,10,1,9,0
98,8329,1,10,1,9,0


Count how many components have a size equal to 1.

In [61]:
query = """
CALL gds.wcc.stream($name) YIELD nodeId, componentId
WITH componentId,  count(*) AS size
WHERE size = 1

RETURN count(componentId) AS nbOfDisconnectedVids
"""

result_data = graph.run(query, name='video_recommendations').to_data_frame()
result_data

,nbOfDisconnectedVids
0,114474


In only 548 disconnected videos tags of Ukraine or Russia were present, but there were about 30 of those per video.

In [62]:
query = """
MATCH (v:video)
WHERE NOT (v:video) -[:recommends]- ()
WITH v

OPTIONAL MATCH (v) -- (t:tag)
WHERE t.title =~ $regex OR t.desc =~ $regex OR t.id =~ $regex OR t.splitTitle =~ $regex
WITH size(collect(distinct(t))) AS countForTags_unique, count(distinct(v)) AS disconnectedVids,
size(collect(t)) AS countForTags

RETURN countForTags, countForTags_unique, disconnectedVids
"""

result_data = graph.run(query, name='video_recommendations', regex = "(?i).*ukrain.*|.*russia.*").to_data_frame()
result_data

,countForTags,countForTags_unique,disconnectedVids
0,16043,548,114474


Count how many of the disconnected videos do not have tags related to Ukraine or Russia.

In [63]:
query = """
MATCH (u:video)
WHERE NOT (u:video) -[:recommends]- ()
WITH u

OPTIONAL MATCH (u) -- (k:tag)
WHERE NOT (k.title =~ $regex OR k.desc =~ $regex OR k.id =~ $regex OR k.splitTitle =~ $regex)

RETURN size(collect(distinct(k))) AS nonUkrTaggedVideos
"""

result_data = graph.run(query, name='video_recommendations', regex = "(?i).*ukrain.*|.*russia.*").to_data_frame()
result_data

,nonUkrTaggedVideos
0,19664


Count how many videos do not have recommendations or tags.

In [64]:
query = """
MATCH (u:video)
WHERE NOT (u:video) -[:recommends]- ()
WITH u

MATCH (u)
WHERE NOT (u) -[:has]- ()
WITH u

RETURN count(distinct(u)) AS NoTagsNoRecs
"""

result_data = graph.run(query, graph = 'video_recommendations').to_data_frame()
result_data

,NoTagsNoRecs
0,2018


Get info about the users who have the largest number of videos that are never recommended.

In [65]:
query = """
MATCH (u:user) -[:author]- (v:video)
WHERE NOT (v:video) -[:recommends]-> ()
OPTIONAL MATCH (v:video) -- (t:tag)
WHERE t.title =~ $regex OR t.desc =~ $regex OR t.id =~ $regex OR t.splitTitle =~ $regex

RETURN distinct(u.id), u.followerCount AS followers, u.diggCount AS likes, u.videoCount AS vidsMade,
count(distinct(v)) AS notRecommendedVids, size(collect(distinct(t))) AS countForTags
ORDER BY notRecommendedVids DESC
LIMIT $limit
"""

result_data = graph.run(query, graph = 'video_recommendations', 
                        limit = 25, regex = "(?i).*ukrain.*|.*russia.*").to_data_frame()
result_data

,(u.id),followers,likes,vidsMade,notRecommendedVids,countForTags
0,ken...demarteau,2800000,16500,2640,93,0
1,d_totti,3200000,5175,447,91,0
2,simonlousberg,1400000,25200,1284,87,2
3,brutofficiel,1600000,413,1533,80,2
4,cbsmornings,801800,33,1475,78,10
5,espn,23600000,3353,6109,76,0
6,raphiquii_,114900,656,3204,70,0
7,mimiermakeup,15100000,11600,674,66,0
8,thieunien.tintuc,903700,14600,295,64,2
9,themsinstem,43800,1387,339,64,28


Get info on the authors of videos that do not recommend videos and have tags related to Ukraine or Russia.

In [71]:
query = """
MATCH (u:user) -[:author]- (v:video)
WHERE (NOT (v:video) -[:recommends]-> ()) AND (u.videoCount > 0)
WITH distinct(u.id) AS ids, u.followerCount AS followers, u.diggCount AS likes, u.videoCount AS vidsMade,
count(distinct(v)) AS notRecommendedVids, v AS videos

OPTIONAL MATCH (videos) -- (t:tag)
WHERE t.title =~ $regex1

RETURN ids, followers, likes, vidsMade, count(distinct(videos)) AS notRecommendedVids, 
size(collect(distinct(t))) AS countForTags_war

ORDER BY notRecommendedVids DESC
LIMIT $limit
"""
regex_1 = "(?i).*ukrain.*|.*russia.*"
regex_2 = "(?i).*war.*|Z|.*putin.*"
result_data = graph.run(query, graph = 'video_recommendations', limit = 25, regex1 = regex_1).to_data_frame()

Calculate what percentage of their videos was not recommended.

In [72]:
result_data["percentage"] = result_data["notRecommendedVids"]/result_data["vidsMade"]
result_data.sort_values(by="percentage", ascending=False)

,ids,followers,likes,vidsMade,notRecommendedVids,countForTags_war,percentage
21,visitukraine_,35900,1504,110,55,14,0.500000
9,thieunien.tintuc,903700,14600,295,64,2,0.216949
1,d_totti,3200000,5175,447,91,0,0.203579
10,themsinstem,43800,1387,339,64,28,0.188791
13,izzy_varane_2,1100000,6152,379,61,0,0.160950
24,franceinfo,281800,16,476,51,2,0.107143
23,melmelmeliciousss,62700,1942,489,52,0,0.106339
19,mrkrul,371400,4677,555,56,0,0.100901
7,mimiermakeup,15100000,11600,674,66,0,0.097923
11,skynews,1100000,35,738,64,9,0.086721


In [73]:
query = """
MATCH (u:user) -[:author]- (v:video)
WHERE NOT (v:video) -[:recommends]-> ()
OPTIONAL MATCH (v:video) -- (t:tag)
WHERE t.title =~ $regex1 OR t.splitTitle =~ $regex1

RETURN distinct(u.id) AS user, u.followerCount AS followers, u.diggCount AS likes, u.videoCount AS vidsMade,
count(distinct(v)) AS notRecommendedVids, count(distinct(t.title)) AS tags_war

ORDER BY tags_war
LIMIT $limit
"""
regex_1 = "(?i).*ukrain.*|.*russia.*"
regex_2 = "(?i)Z|.*putin.*"
result_data = graph.run(query, graph = 'video_recommendations', limit = 50, regex1 = regex_2).to_data_frame()
result_data

,user,followers,likes,vidsMade,notRecommendedVids,tags_war
0,morena.loche,10300,15900,48,28,0
1,kingofharlem,63300,73,96,1,0
2,xacilvo,1114,8821,482,1,0
3,waythis.m,270,105,16,16,0
4,nata_lihka_,526,15500,135,1,0
5,val._.ortiz._,1395,59100,34,1,0
6,istandwithukraine39,10300,5,374,31,0
7,ukraine_best_3,98,5,15,15,0
8,anton_petrus,2953,163,17,1,0
9,squervy,1166,3281,219,1,0


## Users

Create a graph projection for the users and `recommends` relationship.

In [21]:
query = """
CALL gds.graph.project(
    $name, 
    'user', 
    'recommends'
)
YIELD *
"""
graph.run(query, name='user_recommendations')

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
"{user: {label: 'user', properties: {}}}","{recommends: {orientation: 'NATURAL', aggregation: 'DEFAULT', type: 'recommends', properties: {}}}",user_recommendations,127224,191224,625


Get the 100 largest weakly connected components.

In [69]:
query = """
CALL gds.wcc.stream($name) YIELD nodeId, componentId
RETURN componentId, count(*) AS size
ORDER BY size DESC
"""
result_data = graph.run(query, name='user_recommendations').to_data_frame()
result_data

,componentId,size
0,0,109561
1,35,617
2,565,330
3,1177,112
4,1,106
...,...,...
10713,3953,1
10714,3954,1
10715,3956,1
10716,3957,1


Count how many components there are for each size - for some reason a lot of components sized 29, 30, 31.

In [70]:
x = []
i=0
for i in range(1000):
    x.append((i, len(result_data[result_data["size"]==i].value_counts())))
sorted(x, key = lambda x: x[1], reverse = True)

[(1, 10531),
 (31, 77),
 (30, 40),
 (29, 15),
 (33, 8),
 (34, 7),
 (32, 6),
 (35, 6),
 (28, 3),
 (38, 3),
 (36, 2),
 (37, 2),
 (56, 2),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 1),
 (44, 1),
 (46, 1),
 (59, 1),
 (61, 1),
 (79, 1),
 (91, 1),
 (106, 1),
 (112, 1),
 (330, 1),
 (617, 1),
 (0, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 0),
 (6, 0),
 (7, 0),
 (8, 0),
 (9, 0),
 (10, 0),
 (11, 0),
 (12, 0),
 (13, 0),
 (14, 0),
 (15, 0),
 (16, 0),
 (17, 0),
 (18, 0),
 (19, 0),
 (20, 0),
 (21, 0),
 (22, 0),
 (23, 0),
 (24, 0),
 (25, 0),
 (26, 0),
 (27, 0),
 (45, 0),
 (47, 0),
 (48, 0),
 (49, 0),
 (50, 0),
 (51, 0),
 (52, 0),
 (53, 0),
 (54, 0),
 (55, 0),
 (57, 0),
 (58, 0),
 (60, 0),
 (62, 0),
 (63, 0),
 (64, 0),
 (65, 0),
 (66, 0),
 (67, 0),
 (68, 0),
 (69, 0),
 (70, 0),
 (71, 0),
 (72, 0),
 (73, 0),
 (74, 0),
 (75, 0),
 (76, 0),
 (77, 0),
 (78, 0),
 (80, 0),
 (81, 0),
 (82, 0),
 (83, 0),
 (84, 0),
 (85, 0),
 (86, 0),
 (87, 0),
 (88, 0),
 (89, 0),
 (90, 0),
 (92, 0),
 (93, 0),
 (94, 0),
 (95, 0),

For each wcc, get the number of times Ukraine or Russia are mentioned in the user's info.

In [23]:
query = """
// get the components
CALL gds.wcc.stream($name) YIELD nodeId, componentId
WITH componentId,  count(*) AS size, collect(gds.util.asNode(nodeId)) AS nodes

// count the mentions
WITH componentId, size, 
size(
    [node IN nodes WHERE (
        node.uniqueId =~ $regex OR
        node.signature =~ $regex OR
        node.nickname =~ $regex OR
        node.id =~ $regex OR
        node.shortId =~ $regex
    )]
) AS nbMentionUkrainOrRussia

RETURN componentId, size, nbMentionUkrainOrRussia
ORDER BY nbMentionUkrainOrRussia DESC, size DESC
"""
result_data = graph.run(query, name='user_recommendations', regex=".*(?i).*ukrain.*|.*russia.*").to_data_frame()
result_data

,componentId,size,nbMentionUkrainOrRussia
0,0,109561,727
1,1177,112,3
2,1236,31,3
3,35,617,1
4,650,61,1
...,...,...,...
10713,2382,1,0
10714,2383,1,0
10715,2386,1,0
10716,2388,1,0


Get the users that do not recommend anyone.

In [68]:
query = """
MATCH (u:user) -[:author]- (v:video)
WHERE NOT (u:user) -[:recommends]- ()
WITH u,v

RETURN u.id, u.followerCount, u.videoCount, count(v)
LIMIT $limit
"""
result_data = graph.run(query, name = "user_recommendations", limit = 50)
result_data

u.id,u.followerCount,u.videoCount,count(v)
letsparkit,480200,389,30
abramoconstructionsrl.1,3427,34,29
ballforlife41,48200,104,30


# Random walks

## Videos

Random walks are performed starting at videos that mention Ukraine or Russia or for which Ukraine or Russia are mentioned in the tags or songs related to the videos. The walk travels along the `recommends` relationship to other videos. A path of boolean values is recorded for each walk, indicating whether the video at that position in the walk somehow mentions Ukraine or Russia. Only videos in the largest wcc are used, since the 2nd largest does not contain videos that mention Ukraine or Russia and all other wcc's have a size smaller than or equal to 2.

In [24]:
query = """
// get the videos in the largest wcc
CALL gds.wcc.stream($name) YIELD nodeId, componentId
WITH componentId,  count(*) AS compSize, collect(gds.util.asNode(nodeId)) AS videosLargestComp
WITH componentId, compSize, videosLargestComp
ORDER BY compSize DESC LIMIT 1

// get the users from which the random walk should start
UNWIND videosLargestComp AS v
WITH v, compSize
OPTIONAL MATCH (t:tag)--(v)
WITH v, t, compSize
OPTIONAL MATCH (s:song)--(v)
WITH v, t, s, compSize
WHERE (
        v.desc =~ $regex OR 
        v.stickerText =~ $regex
    ) OR (
        t.title =~ $regex OR 
        t.desc =~ $regex OR 
        t.id =~ $regex OR 
        t.splitTitle =~ $regex
    ) OR (
        s.id =~ $regex OR 
        s.title =~ $regex OR 
        s.album =~ $regex
    )    
WITH collect(v) AS sourceNodes, compSize

// perform the random walks
CALL gds.beta.randomWalk.stream(
  $name,
  {
  sourceNodes: sourceNodes,
  walkLength: $walkLength,
  walksPerNode: $walksPerNode
  }
) YIELD nodeIds, path
WITH nodeIds AS videoIDs, compSize
WITH videoIDs, size(videoIDs) AS pathlength, compSize
ORDER BY pathlength DESC

WITH videoIDs, pathlength, compSize

// return a boolean list for whether the video at the same position in the path 
// mentions Ukraine or Russia 

// read the videoIDs of the path as rows
UNWIND videoIDs AS videoID

// convert the IDs to nodes
WITH videoIDs, pathlength, gds.util.asNode(videoID) AS v, compSize

// if the video itself mentions Ukraine or Russia, total > 0
WITH videoIDs, pathlength, compSize, v,
CASE
    WHEN (
        v.desc =~ $regex OR 
        v.stickerText =~ $regex
    ) 
    THEN 1
    ELSE 0
END AS total

// get the tags of the video if there are any
OPTIONAL MATCH (t:tag)--(v)

// if the tags mention Ukraine or Russia, sum(countTags) > 0
WITH videoIDs, pathlength, compSize, v, total,
CASE
    WHEN (
        t.title =~ $regex OR 
        t.desc =~ $regex OR 
        t.id =~ $regex OR 
        t.splitTitle =~ $regex
    ) 
    THEN 1
    ELSE 0
END AS countTags
WITH videoIDs, pathlength, compSize, v, total + sum(countTags) AS total

// get the songs of the video if there are any
OPTIONAL MATCH (s:song)--(v)

// if the songs mention Ukraine or Russia, sum(countSongs) > 0
WITH videoIDs, pathlength, compSize, v, total,
CASE
    WHEN (
        s.id =~ $regex OR 
        s.title =~ $regex OR 
        s.album =~ $regex
    ) 
    THEN 1
    ELSE 0
END AS countSongs
WITH videoIDs, pathlength, compSize, v, total + sum(countSongs) AS total

// collect the totals per path and convert to boolean, if total > 0 then 
// either the videos, tags, or songs mention Ukraine or Russia
WITH videoIDs, pathlength, compSize,
COLLECT(
CASE
    WHEN (total > 0)
    THEN true
    ELSE false
END
) AS relatedToUkrOrRussia

RETURN videoIDs, pathlength, relatedToUkrOrRussia, compSize
"""

wLength = 15
result_data = graph.run(query, name='video_recommendations', regex='(?i).*ukrain.*|.*russia.*', 
                        walkLength=wLength, walksPerNode=100).to_data_frame()
result_data

,videoIDs,pathlength,relatedToUkrOrRussia,compSize
0,"[2433, 1951, 1960, 1964, 2570, 9069, 7699]",7,"[True, False, False, False, False, False, False]",22800
1,"[533, 1918, 1911, 1699, 2769, 8985]",6,"[True, False, False, False, False, False]",22800
2,"[611, 2657, 1631, 1629, 1918, 3860]",6,"[True, False, False, False, False, False]",22800
3,"[177, 1763, 1921, 12093, 1522, 5418]",6,"[True, False, False, False, False, False]",22800
4,"[816, 1609, 1618, 1667, 2714, 5973]",6,"[True, False, False, False, False, False]",22800
...,...,...,...,...
20315,"[5, 19205]",2,"[True, False]",22800
20316,"[5, 19499]",2,"[True, False]",22800
20317,"[5, 15262]",2,"[True, False]",22800
20318,"[5, 8373]",2,"[True, False]",22800


If the boolean path is shorter than the path of visisted video IDs, this means that at least one video was visited more than once during the walk.

In [25]:
boolPath_too_short = result_data[['pathlength','relatedToUkrOrRussia']].apply(
    lambda row: len(row['relatedToUkrOrRussia']) != row['pathlength'], axis=1)

print('paths with revisited nodes:\n')
result_data.loc[boolPath_too_short,:]

paths with revisited nodes:



,videoIDs,pathlength,relatedToUkrOrRussia,compSize


For each boolean path, get the first occurence of `False`. This is the first video that did not mention Ukraine or Russia in some way. This will be used to calculate the mean number of steps along the recommendations it takes to reach a video that is not related to Ukraine or Russia (when starting from a video that does mention Ukraine or Russia). This number of steps will be refered to as the mean escape length.

In [26]:
result_data['index_first_False'] = result_data['relatedToUkrOrRussia'].apply(lambda row: get_index_first_False(row))
result_data.sort_values(by=["pathlength", "index_first_False"], ascending=False)

,videoIDs,pathlength,relatedToUkrOrRussia,compSize,index_first_False
0,"[2433, 1951, 1960, 1964, 2570, 9069, 7699]",7,"[True, False, False, False, False, False, False]",22800,1
1,"[533, 1918, 1911, 1699, 2769, 8985]",6,"[True, False, False, False, False, False]",22800,1
2,"[611, 2657, 1631, 1629, 1918, 3860]",6,"[True, False, False, False, False, False]",22800,1
3,"[177, 1763, 1921, 12093, 1522, 5418]",6,"[True, False, False, False, False, False]",22800,1
4,"[816, 1609, 1618, 1667, 2714, 5973]",6,"[True, False, False, False, False, False]",22800,1
...,...,...,...,...,...
20315,"[5, 19205]",2,"[True, False]",22800,1
20316,"[5, 19499]",2,"[True, False]",22800,1
20317,"[5, 15262]",2,"[True, False]",22800,1
20318,"[5, 8373]",2,"[True, False]",22800,1


If many boolean paths never contain `False` and have the maximum pathlength possible, `walkLength` should be set to a higher value, otherwise the mean escape length will seem shorter than it actually is.

In [27]:
never_False = result_data.loc[(result_data.index_first_False == result_data.pathlength) & (
result_data.pathlength == wLength), :]
print(f'\nnb rows: {len(never_False)}')
never_False


nb rows: 0


,videoIDs,pathlength,relatedToUkrOrRussia,compSize,index_first_False


The number of steps that are taken before you reach a video that does not mention Ukraine or Russia is the index of the first `False` calculated earlier minus one.

In [28]:
result_data['steps_until_False'] = result_data['index_first_False'].apply(lambda row: row-1)
result_data.sort_values(by=["pathlength","index_first_False"], ascending=False)

,videoIDs,pathlength,relatedToUkrOrRussia,compSize,index_first_False,steps_until_False
0,"[2433, 1951, 1960, 1964, 2570, 9069, 7699]",7,"[True, False, False, False, False, False, False]",22800,1,0
1,"[533, 1918, 1911, 1699, 2769, 8985]",6,"[True, False, False, False, False, False]",22800,1,0
2,"[611, 2657, 1631, 1629, 1918, 3860]",6,"[True, False, False, False, False, False]",22800,1,0
3,"[177, 1763, 1921, 12093, 1522, 5418]",6,"[True, False, False, False, False, False]",22800,1,0
4,"[816, 1609, 1618, 1667, 2714, 5973]",6,"[True, False, False, False, False, False]",22800,1,0
...,...,...,...,...,...,...
20315,"[5, 19205]",2,"[True, False]",22800,1,0
20316,"[5, 19499]",2,"[True, False]",22800,1,0
20317,"[5, 15262]",2,"[True, False]",22800,1,0
20318,"[5, 8373]",2,"[True, False]",22800,1,0


In [29]:
result_data.sort_values(by=["index_first_False","pathlength"], ascending=False)

,videoIDs,pathlength,relatedToUkrOrRussia,compSize,index_first_False,steps_until_False
26,"[543, 1952, 3285, 1752, 6297]",5,"[True, True, False, False, False]",22800,2,1
157,"[543, 1952, 3593, 14762]",4,"[True, True, False, False]",22800,2,1
158,"[543, 1952, 2637, 15870]",4,"[True, True, False, False]",22800,2,1
160,"[543, 1952, 2637, 23485]",4,"[True, True, False, False]",22800,2,1
161,"[543, 1952, 3285, 3352]",4,"[True, True, False, False]",22800,2,1
...,...,...,...,...,...,...
20315,"[5, 19205]",2,"[True, False]",22800,1,0
20316,"[5, 19499]",2,"[True, False]",22800,1,0
20317,"[5, 15262]",2,"[True, False]",22800,1,0
20318,"[5, 8373]",2,"[True, False]",22800,1,0


In [30]:
max_pathlength = result_data['pathlength'].max()
min_path_length = 2
print(f'maximum pathlength found:       {max_pathlength}')
print(f'minimum pathlength considered:  {min_path_length}\n')

steps_taken_until_False = result_data.loc[result_data['pathlength'] >= min_path_length, 'steps_until_False']
print(f'min nb of steps until False:    {steps_taken_until_False.min():.0f}')
print(f'avg nb of steps until False:    {steps_taken_until_False.mean():.5f}')
print(f'median nb of steps until False: {steps_taken_until_False.median():.0f}')
print(f'max nb of steps until False:    {steps_taken_until_False.max():.0f}')

maximum pathlength found:       7
minimum pathlength considered:  2

min nb of steps until False:    0
avg nb of steps until False:    0.00487
median nb of steps until False: 0
max nb of steps until False:    1


In [31]:
nb = len(result_data.loc[result_data['steps_until_False'] > 1, :])
print(f'nb of walks with a "nb of steps until False" greater than 1: {nb}')

nb of walks with a "nb of steps until False" greater than 1: 0


In [32]:
unique_paths = result_data['videoIDs'].apply(lambda row: str(row))
print(f'nb of unique paths: {len(set(unique_paths.values))}')
print(f'total nb of paths:  {len(result_data)}')

nb of unique paths: 20320
total nb of paths:  20320


### Closer look at paths with longer escape length

Paths where at least the first 2 nodes are related to Ukraine or Russia.

In [33]:
result_data.loc[result_data['steps_until_False'] == 1, :]

,videoIDs,pathlength,relatedToUkrOrRussia,compSize,index_first_False,steps_until_False
26,"[543, 1952, 3285, 1752, 6297]",5,"[True, True, False, False, False]",22800,2,1
157,"[543, 1952, 3593, 14762]",4,"[True, True, False, False]",22800,2,1
158,"[543, 1952, 2637, 15870]",4,"[True, True, False, False]",22800,2,1
160,"[543, 1952, 2637, 23485]",4,"[True, True, False, False]",22800,2,1
161,"[543, 1952, 3285, 3352]",4,"[True, True, False, False]",22800,2,1
...,...,...,...,...,...,...
19203,"[211, 2011]",2,"[True, True]",22800,2,1
19847,"[35, 19206]",2,"[True, True]",22800,2,1
19858,"[40, 4493]",2,"[True, True]",22800,2,1
19990,"[10, 7610]",2,"[True, True]",22800,2,1


Get the paths.

In [34]:
t = result_data.loc[result_data['steps_until_False'] == 1, :]
paths_of_interest = list(t["videoIDs"].values)
print(paths_of_interest)

[[543, 1952, 3285, 1752, 6297], [543, 1952, 3593, 14762], [543, 1952, 2637, 15870], [543, 1952, 2637, 23485], [543, 1952, 3285, 3352], [543, 1952, 2637, 8267], [543, 1952, 3285, 6285], [170, 269, 2547, 9380], [253, 1952, 3593, 14762], [253, 1952, 7032], [543, 1952, 12102], [543, 1952, 7032], [543, 1952, 3472], [259, 1968, 10577], [642, 1968, 10577], [642, 1968, 24226], [642, 1968, 15478], [152, 361, 20013], [19, 6531, 17650], [352, 12866, 21289], [170, 269, 4145], [396, 1968, 20115], [396, 1968, 14054], [396, 1968, 15478], [396, 1968, 24226], [396, 1968, 10577], [1397, 1952, 12102], [251, 1940, 17932], [53, 24118], [114, 10085], [27, 6999], [31, 12850], [3792, 2737], [1, 18761], [1, 3148], [192, 12988], [2449, 23061], [90, 12595], [51, 12816], [94, 14114], [1397, 8876], [966, 20831], [963, 16051], [181, 14165], [959, 8450], [960, 962], [921, 14165], [18, 2441], [14, 962], [47, 19206], [171, 10599], [45, 15939], [47, 12300], [368, 962], [844, 6177], [87, 18987], [87, 22271], [841, 14717

Get all the nodes on these paths.

In [35]:
query = """
WITH $paths AS paths
UNWIND paths AS path
UNWIND path AS nodeId
RETURN gds.util.asNode(nodeId)
"""

result_data = graph.run(query, paths=paths_of_interest).to_data_frame()
result_data

,gds.util.asNode(nodeId)
0,"{'shareEnabled': 1, 'showNotPass': 0, 'private..."
1,"{'shareEnabled': 1, 'showNotPass': 0, 'private..."
2,"{'shareEnabled': 1, 'showNotPass': 0, 'private..."
3,"{'shareEnabled': 1, 'showNotPass': 0, 'private..."
4,"{'shareEnabled': 1, 'showNotPass': 0, 'private..."
...,...
231,"{'effectStickers': 'Green Screen', 'shareEnabl..."
232,"{'shareEnabled': 1, 'showNotPass': 0, 'private..."
233,"{'shareEnabled': 1, 'showNotPass': 0, 'private..."
234,"{'shareEnabled': 1, 'showNotPass': 0, 'private..."


## Users

Random walks are performed starting at users that mention Ukraine or Russia or for which Ukraine or Russia are mentioned in the videos of the user, or in the tags and songs related to these videos. The walk travels along the `recommends` relationship to other users. A path of boolean values is recorded for each walk, indicating whether the user at that position in the walk somehow mentions Ukraine or Russia. Due to the large number of source nodes that is found, a random samlpe is selected.

In [36]:
query = """
// do random walks starting at users who mention Ukraine or Russia, or whos videos or the related tags and songs mention
// Ukraine or Russia
// get the users from which to start walking
MATCH (u:user)--(v:video)
WITH u, v
OPTIONAL MATCH (t:tag)--(v)
WITH u, v, t
OPTIONAL MATCH (s:song)--(v)
WITH u, v, t, s
WHERE (
        v.desc =~ $regex OR 
        v.stickerText =~ $regex
    ) OR (
        t.title =~ $regex OR 
        t.desc =~ $regex OR 
        t.id =~ $regex OR 
        t.splitTitle =~ $regex OR
        toLower(t.title) IN $tags        
    ) OR (
        s.id =~ $regex OR 
        s.title =~ $regex OR 
        s.album =~ $regex
    ) OR (
        u.uniqueId =~ $regex OR
        u.signature =~ $regex OR
        u.nickname =~ $regex OR
        u.id =~ $regex OR
        u.shortId =~ $regex
    )
    
// randomly select a subset of nodes, to reduce computation time 
WITH u
WHERE rand() <= $p
WITH COLLECT(u) AS sourceNodes
WITH sourceNodes, size(sourceNodes) AS nbSourceNodes

CALL gds.beta.randomWalk.stream(
  $name,
  {
  sourceNodes: sourceNodes,
  walkLength: $walkLength,
  walksPerNode: $walksPerNode,
  returnFactor: $returnFactor
  }
) YIELD nodeIds, path
WITH nodeIds AS userIDs, nbSourceNodes
WITH userIDs, nbSourceNodes

WITH userIDs, nbSourceNodes


// return a list with True and False for whether the user at the same position in the path is related
// to Ukraine or Russia 

// read the elements of the list as rows
UNWIND userIDs AS userID
// convert the IDs to nodes
WITH userIDs, gds.util.asNode(userID) AS u, nbSourceNodes

// if the user itself is related to Ukraine or Russia, total > 0
WITH userIDs, u, nbSourceNodes,
CASE
    WHEN (
        u.uniqueId =~ $regex OR
        u.signature =~ $regex OR
        u.nickname =~ $regex OR
        u.id =~ $regex OR
        u.shortId =~ $regex
    ) 
    THEN 1
    ELSE 0
END AS total

// get the videos
OPTIONAL MATCH (v:video)--(u)
// if the videos are related to Ukraine or Russia, sum(countVids) > 0
WITH userIDs, u, v, nbSourceNodes, total,
CASE
    WHEN (
        v.desc =~ $regex OR 
        v.stickerText =~ $regex
    ) 
    THEN 1
    ELSE 0
END AS countVids
WITH userIDs, u, v, nbSourceNodes, total, countVids

// get the tags
OPTIONAL MATCH (t:tag)--(v)
// if the tags are related to Ukraine or Russia, sum(countTags) > 0
WITH userIDs, u, v, nbSourceNodes, total, countVids,
CASE
    WHEN (
        t.title =~ $regex OR 
        t.desc =~ $regex OR 
        t.id =~ $regex OR 
        t.splitTitle =~ $regex OR
        toLower(t.title) IN $tags 
    ) 
    THEN 1
    ELSE 0
END AS countTags
WITH userIDs, u, v, nbSourceNodes, total, countVids, countTags

// get the songs
OPTIONAL MATCH (s:song)--(v)
// if the songs are related to Ukraine or Russia, sum(countSongs) > 0
WITH userIDs, u, v, nbSourceNodes, total, countVids, countTags,
CASE
    WHEN (
        s.id =~ $regex OR 
        s.title =~ $regex OR 
        s.album =~ $regex
    ) 
    THEN 1
    ELSE 0
END AS countSongs
WITH userIDs, u, nbSourceNodes, total, countVids, countTags, countSongs

// sum all the counts to get the total per user
WITH userIDs, u, nbSourceNodes, total, 
    sum(countVids) AS sumVids, 
    sum(countTags) AS sumTags, 
    sum(countSongs) AS sumSongs
WITH userIDs, u, nbSourceNodes, total + sumVids + sumTags + sumSongs AS total

// convert to boolean
WITH userIDs, u, nbSourceNodes, 
CASE
    WHEN (total > 0)
    THEN true
    ELSE false
END AS boolInd

// collect the boolean values per path
WITH userIDs, nbSourceNodes, collect(boolInd) AS relatedToUkrOrRussia
RETURN userIDs, size(userIDs) AS pathlength, relatedToUkrOrRussia, nbSourceNodes
ORDER BY pathlength DESC

"""

tags_list = ["ukraine", "russia", "ukrainewar", "ukrainerussiaconflict", "ukrainewarrussia", "supportukraine", 
             "ukrainevsrussia", "prayforukraine", "russianarmy", "z"]

wLength = 15
# the returnFactor is the tendency to revisit the last visited node, the higher this value, the lower the tendency
result_data = graph.run(query, name='user_recommendations', regex='(?i).*ukrain.*|.*russia.*', 
                        walkLength=wLength, walksPerNode=5, p=0.5, tags=tags_list, returnFactor=2).to_data_frame()
result_data

,userIDs,pathlength,relatedToUkrOrRussia,nbSourceNodes
0,"[137795, 197194, 137381, 137330, 154346, 13853...",9,"[True, False, True, True, False, True, True, F...",28917
1,"[151499, 154348, 154349, 153284, 183592, 16723...",9,"[True, False, False, True, False, False, True,...",28917
2,"[153282, 192276, 151501, 157324, 157914, 15149...",8,"[True, False, True, False, False, True, False,...",28917
3,"[137916, 157780, 154108, 161241, 199051, 18806...",8,"[True, False, False, False, True, False, True,...",28917
4,"[137320, 153233, 160260, 177939, 191403, 16456...",8,"[True, True, False, False, False, False, False...",28917
...,...,...,...,...
3993,"[138379, 202290]",2,"[True, False]",28917
3994,"[138379, 153945]",2,"[True, False]",28917
3995,"[138379, 172502]",2,"[True, False]",28917
3996,"[138379, 176289]",2,"[True, False]",28917


If the boolean path is shorter than the path of visisted user IDs, this means that at least one user was visited more than once during the walk.

In [38]:
boolPath_too_short = result_data[['pathlength','relatedToUkrOrRussia']].apply(
    lambda row: len(row['relatedToUkrOrRussia']) != row['pathlength'], axis=1)

print('paths with revisited nodes:\n')
result_data.loc[boolPath_too_short,:]

paths with revisited nodes:



,userIDs,pathlength,relatedToUkrOrRussia,nbSourceNodes
1,"[151499, 154348, 154349, 153284, 183592, 16723...",9,"[True, False, False, True, False, False, True,...",28917
6,"[137377, 192478, 137377, 137374, 187285, 17972...",7,"[True, False, True, False, False, False]",28917
14,"[151498, 154349, 154341, 151498, 199010, 252787]",6,"[True, False, False, False, True]",28917
19,"[137870, 171801, 137870, 153626, 177874, 254295]",6,"[True, False, True, False, False]",28917
24,"[137320, 153231, 161452, 153231, 155725, 232283]",6,"[True, True, True, False, True]",28917
45,"[138045, 147955, 140895, 147955, 179824]",5,"[True, False, False, False]",28917
46,"[138087, 187418, 163375, 138087, 167707]",5,"[True, False, False, False]",28917
52,"[138403, 174506, 137742, 174506, 180726]",5,"[True, False, True, False]",28917
66,"[153409, 155156, 153409, 155160, 156719]",5,"[True, False, False, False]",28917
72,"[137353, 154771, 137353, 154779, 157106]",5,"[True, False, False, False]",28917


Correct the paths where nodes were revisited.

In [43]:
result_data.loc[boolPath_too_short, 'relatedToUkrOrRussia_corrected'] = result_data.loc[
    boolPath_too_short, ['userIDs','relatedToUkrOrRussia']].apply(
    lambda row: deal_with_revisit(row), axis=1)
result_data.loc[~boolPath_too_short, 'relatedToUkrOrRussia_corrected'] = result_data.loc[
    ~boolPath_too_short, 'relatedToUkrOrRussia']
result_data.loc[boolPath_too_short,:]

,userIDs,pathlength,relatedToUkrOrRussia,nbSourceNodes,index_first_False,relatedToUkrOrRussia_corrected
1,"[151499, 154348, 154349, 153284, 183592, 16723...",9,"[True, False, False, True, False, False, True,...",28917,1,"[True, False, False, True, False, False, True,..."
6,"[137377, 192478, 137377, 137374, 187285, 17972...",7,"[True, False, True, False, False, False]",28917,1,"[True, False, True, True, False, False, False]"
14,"[151498, 154349, 154341, 151498, 199010, 252787]",6,"[True, False, False, False, True]",28917,1,"[True, False, False, True, False, True]"
19,"[137870, 171801, 137870, 153626, 177874, 254295]",6,"[True, False, True, False, False]",28917,1,"[True, False, True, True, False, False]"
24,"[137320, 153231, 161452, 153231, 155725, 232283]",6,"[True, True, True, False, True]",28917,4,"[True, True, True, True, False, True]"
45,"[138045, 147955, 140895, 147955, 179824]",5,"[True, False, False, False]",28917,1,"[True, False, False, False, False]"
46,"[138087, 187418, 163375, 138087, 167707]",5,"[True, False, False, False]",28917,1,"[True, False, False, True, False]"
52,"[138403, 174506, 137742, 174506, 180726]",5,"[True, False, True, False]",28917,1,"[True, False, True, False, False]"
66,"[153409, 155156, 153409, 155160, 156719]",5,"[True, False, False, False]",28917,1,"[True, False, True, False, False]"
72,"[137353, 154771, 137353, 154779, 157106]",5,"[True, False, False, False]",28917,1,"[True, False, True, False, False]"


For each boolean path, get the first occurence of `False`. This is the first user that did not mention Ukraine or Russia in some way. This will be used to calculate the mean number of steps along the recommendations it takes to reach a user that is not related to Ukraine or Russia (when starting from a user that does mention Ukraine or Russia). This number of steps will be refered to as the mean escape length.

In [44]:
result_data['index_first_False'] = result_data['relatedToUkrOrRussia_corrected'].apply(lambda row: get_index_first_False(row))
result_data.sort_values(by=["pathlength", "index_first_False"], ascending=False)

,userIDs,pathlength,relatedToUkrOrRussia,nbSourceNodes,index_first_False,relatedToUkrOrRussia_corrected
0,"[137795, 197194, 137381, 137330, 154346, 13853...",9,"[True, False, True, True, False, True, True, F...",28917,1,"[True, False, True, True, False, True, True, F..."
1,"[151499, 154348, 154349, 153284, 183592, 16723...",9,"[True, False, False, True, False, False, True,...",28917,1,"[True, False, False, True, False, False, True,..."
4,"[137320, 153233, 160260, 177939, 191403, 16456...",8,"[True, True, False, False, False, False, False...",28917,2,"[True, True, False, False, False, False, False..."
2,"[153282, 192276, 151501, 157324, 157914, 15149...",8,"[True, False, True, False, False, True, False,...",28917,1,"[True, False, True, False, False, True, False,..."
3,"[137916, 157780, 154108, 161241, 199051, 18806...",8,"[True, False, False, False, True, False, True,...",28917,1,"[True, False, False, False, True, False, True,..."
...,...,...,...,...,...,...
3993,"[138379, 202290]",2,"[True, False]",28917,1,"[True, False]"
3994,"[138379, 153945]",2,"[True, False]",28917,1,"[True, False]"
3995,"[138379, 172502]",2,"[True, False]",28917,1,"[True, False]"
3996,"[138379, 176289]",2,"[True, False]",28917,1,"[True, False]"


If many boolean paths never contain `False` and have the maximum pathlength possible, `walkLength` should be set to a higher value, otherwise the mean escape length will seem shorter than it actually is.

In [45]:
never_False = result_data.loc[(result_data.index_first_False == result_data.pathlength) & (
result_data.pathlength == wLength), :]
print(f'\nnb rows: {len(never_False)}')
never_False


nb rows: 0


,userIDs,pathlength,relatedToUkrOrRussia,nbSourceNodes,index_first_False,relatedToUkrOrRussia_corrected


The number of steps that are taken before you reach a user that does not mention Ukraine or Russia is the index of the first `False` calculated earlier minus one.

In [46]:
result_data['steps_until_False'] = result_data['index_first_False'].apply(lambda row: row-1)
result_data.sort_values(by=["pathlength","index_first_False"], ascending=False)

,userIDs,pathlength,relatedToUkrOrRussia,nbSourceNodes,index_first_False,relatedToUkrOrRussia_corrected,steps_until_False
0,"[137795, 197194, 137381, 137330, 154346, 13853...",9,"[True, False, True, True, False, True, True, F...",28917,1,"[True, False, True, True, False, True, True, F...",0
1,"[151499, 154348, 154349, 153284, 183592, 16723...",9,"[True, False, False, True, False, False, True,...",28917,1,"[True, False, False, True, False, False, True,...",0
4,"[137320, 153233, 160260, 177939, 191403, 16456...",8,"[True, True, False, False, False, False, False...",28917,2,"[True, True, False, False, False, False, False...",1
2,"[153282, 192276, 151501, 157324, 157914, 15149...",8,"[True, False, True, False, False, True, False,...",28917,1,"[True, False, True, False, False, True, False,...",0
3,"[137916, 157780, 154108, 161241, 199051, 18806...",8,"[True, False, False, False, True, False, True,...",28917,1,"[True, False, False, False, True, False, True,...",0
...,...,...,...,...,...,...,...
3993,"[138379, 202290]",2,"[True, False]",28917,1,"[True, False]",0
3994,"[138379, 153945]",2,"[True, False]",28917,1,"[True, False]",0
3995,"[138379, 172502]",2,"[True, False]",28917,1,"[True, False]",0
3996,"[138379, 176289]",2,"[True, False]",28917,1,"[True, False]",0


In [47]:
result_data.sort_values(by=["index_first_False","pathlength"], ascending=False)

,userIDs,pathlength,relatedToUkrOrRussia,nbSourceNodes,index_first_False,relatedToUkrOrRussia_corrected,steps_until_False
24,"[137320, 153231, 161452, 153231, 155725, 232283]",6,"[True, True, True, False, True]",28917,4,"[True, True, True, True, False, True]",3
17,"[153297, 174662, 138018, 162460, 180824, 240099]",6,"[True, True, True, False, False, False]",28917,3,"[True, True, True, False, False, False]",2
43,"[138025, 151497, 153282, 167235, 215828]",5,"[True, True, True, False, False]",28917,3,"[True, True, True, False, False]",2
74,"[137374, 137377, 153305, 155111, 252449]",5,"[True, True, True, False, False]",28917,3,"[True, True, True, False, False]",2
78,"[137405, 137871, 137405, 185782, 155466]",5,"[True, True, False, False]",28917,3,"[True, True, True, False, False]",2
...,...,...,...,...,...,...,...
3993,"[138379, 202290]",2,"[True, False]",28917,1,"[True, False]",0
3994,"[138379, 153945]",2,"[True, False]",28917,1,"[True, False]",0
3995,"[138379, 172502]",2,"[True, False]",28917,1,"[True, False]",0
3996,"[138379, 176289]",2,"[True, False]",28917,1,"[True, False]",0


In [48]:
max_pathlength = result_data['pathlength'].max()
min_path_length = 2
print(f'maximum pathlength found:       {max_pathlength}')
print(f'minimum pathlength considered:  {min_path_length}\n')

steps_taken_until_False = result_data.loc[result_data['pathlength'] >= min_path_length, 'steps_until_False']
print(f'min nb of steps until False:    {steps_taken_until_False.min():.0f}')
print(f'avg nb of steps until False:    {steps_taken_until_False.mean():.5f}')
print(f'median nb of steps until False: {steps_taken_until_False.median():.0f}')
print(f'max nb of steps until False:    {steps_taken_until_False.max():.0f}')

maximum pathlength found:       9
minimum pathlength considered:  2

min nb of steps until False:    0
avg nb of steps until False:    0.06603
median nb of steps until False: 0
max nb of steps until False:    3


In [49]:
nb = len(result_data.loc[result_data['steps_until_False'] > 1, :])
print(f'nb of walks with a "nb of steps until False" greater than 1: {nb}')

nb of walks with a "nb of steps until False" greater than 1: 27


In [50]:
unique_paths = result_data['userIDs'].apply(lambda row: str(row))
print(f'nb of unique paths: {len(set(unique_paths.values))}')
print(f'total nb of paths:  {len(result_data)}')

nb of unique paths: 3998
total nb of paths:  3998


### Closer look at paths with longer escape length

Paths where at least the first 3 nodes are related to Ukraine or Russia.

In [51]:
result_data.loc[result_data['steps_until_False'] > 1, :]

,userIDs,pathlength,relatedToUkrOrRussia,nbSourceNodes,index_first_False,relatedToUkrOrRussia_corrected,steps_until_False
17,"[153297, 174662, 138018, 162460, 180824, 240099]",6,"[True, True, True, False, False, False]",28917,3,"[True, True, True, False, False, False]",2
24,"[137320, 153231, 161452, 153231, 155725, 232283]",6,"[True, True, True, False, True]",28917,4,"[True, True, True, True, False, True]",3
43,"[138025, 151497, 153282, 167235, 215828]",5,"[True, True, True, False, False]",28917,3,"[True, True, True, False, False]",2
74,"[137374, 137377, 153305, 155111, 252449]",5,"[True, True, True, False, False]",28917,3,"[True, True, True, False, False]",2
78,"[137405, 137871, 137405, 185782, 155466]",5,"[True, True, False, False]",28917,3,"[True, True, True, False, False]",2
96,"[137335, 151511, 138413, 261782]",4,"[True, True, True, False]",28917,3,"[True, True, True, False]",2
116,"[137327, 153268, 154288, 155821]",4,"[True, True, True, False]",28917,3,"[True, True, True, False]",2
128,"[137369, 151597, 137369, 197596]",4,"[True, True, False]",28917,3,"[True, True, True, False]",2
137,"[137404, 154102, 138262, 181060]",4,"[True, True, True, False]",28917,3,"[True, True, True, False]",2
154,"[137700, 151499, 137330, 154344]",4,"[True, True, True, False]",28917,3,"[True, True, True, False]",2


Get the paths.

In [52]:
t = result_data.loc[result_data['steps_until_False'] > 1, :]
paths_of_interest = list(t["userIDs"].values)
print(paths_of_interest)

[[153297, 174662, 138018, 162460, 180824, 240099], [137320, 153231, 161452, 153231, 155725, 232283], [138025, 151497, 153282, 167235, 215828], [137374, 137377, 153305, 155111, 252449], [137405, 137871, 137405, 185782, 155466], [137335, 151511, 138413, 261782], [137327, 153268, 154288, 155821], [137369, 151597, 137369, 197596], [137404, 154102, 138262, 181060], [137700, 151499, 137330, 154344], [137742, 195951, 153282, 169232], [138262, 199051, 195390, 157009], [138435, 199051, 138435, 176736], [138632, 138403, 199051, 159523], [151501, 154343, 153285, 247039], [153407, 137330, 151498, 225518], [153255, 137388, 153304, 263169], [153996, 137399, 199051, 161551], [153417, 137375, 151610, 231024], [149858, 139105, 145816], [138062, 151468, 171998], [137343, 151537, 186869], [137327, 153268, 153811], [137451, 138265, 247719], [137491, 138275, 151835], [137742, 138339, 252575], [138590, 153297, 153790]]


Get all the nodes on these paths.

In [53]:
query = """
WITH $paths AS paths
UNWIND paths AS path
UNWIND path AS nodeId
RETURN gds.util.asNode(nodeId)
"""

result_data = graph.run(query, paths=paths_of_interest).to_data_frame()
result_data

,gds.util.asNode(nodeId)
0,"{'shortId': '', 'isADVirtual': 0, 'signature':..."
1,"{'shortId': '', 'isADVirtual': 0, 'avatarMediu..."
2,"{'shortId': '', 'isADVirtual': 0, 'signature':..."
3,"{'shortId': '', 'isADVirtual': 0, 'avatarMediu..."
4,"{'shortId': '', 'isADVirtual': 0, 'avatarMediu..."
...,...
102,"{'shortId': '', 'isADVirtual': 0, 'signature':..."
103,"{'shortId': '', 'isADVirtual': 0, 'signature':..."
104,"{'shortId': '', 'isADVirtual': 0, 'signature':..."
105,"{'shortId': '', 'isADVirtual': 0, 'signature':..."


Make a graph projection of the nodes and the recommendations between the nodes.

In [54]:
query = """
CALL gds.graph.project.cypher(
  'longest_paths',
  'WITH $paths AS paths
   UNWIND paths AS path
   UNWIND path AS nodeId
   RETURN nodeId AS id',
   
  'WITH $paths AS paths
   UNWIND paths AS path
   WITH path
   MATCH (n:user)-[r:recommends]->(m:user) 
   WHERE id(n) IN path AND id(m) IN path
   RETURN id(n) AS source, id(m) AS target',
  
  { parameters: { paths: $paths} }
  )
YIELD
  graphName AS graph, nodeQuery, nodeCount AS nodes, relationshipQuery, relationshipCount AS rels
"""

result_data = graph.run(query, paths=paths_of_interest).to_data_frame()
result_data

,graph,nodeQuery,nodes,relationshipQuery,rels
0,longest_paths,WITH $paths AS paths\n UNWIND paths AS path\...,93,WITH $paths AS paths\n UNWIND paths AS path\...,125


Calculate centrality measures on the graph projection.

In [55]:
query = """
CALL gds.degree.stream('longest_paths')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).uniqueId AS name, score AS connections
ORDER BY connections DESC
"""
# in-degree
result_data = graph.run(query).to_data_frame()
result_data

,name,connections
0,watson_actu,6.0
1,russianvsukrainianwar,5.0
2,pray.for.the.world.2022,4.0
3,24news4u,4.0
4,city__resident,4.0
...,...,...
88,bestpage22,0.0
89,supportukraineastopwar,0.0
90,diyabutt20,0.0
91,gvc_gary,0.0


In [56]:
query = """
CALL gds.degree.stream('longest_paths', { orientation: 'REVERSE' })
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).uniqueId AS name, score AS connections
ORDER BY connections DESC
"""
# out-degree
result_data = graph.run(query).to_data_frame()
result_data

,name,connections
0,russianvsukrainianwar,6.0
1,24news4u,4.0
2,eliemudiayi,4.0
3,pray.for.the.world.2022,3.0
4,now_live,3.0
...,...,...
88,treyyingst,0.0
89,on.dahak,0.0
90,bbcnews,0.0
91,visitukraine_,0.0


In [57]:
query = """
CALL gds.beta.closeness.stream('longest_paths')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).uniqueId AS name, score AS centrality
ORDER BY centrality DESC;
"""
# default clonseness
result_data = graph.run(query).to_data_frame()
result_data

,name,centrality
0,pray.for.the.world.2022,1.0
1,forcesnews,1.0
2,itsrachelscott,1.0
3,watson_actu,1.0
4,timonthegram,1.0
...,...,...
88,treyyingst,0.0
89,on.dahak,0.0
90,bbcnews,0.0
91,visitukraine_,0.0


In [58]:
query = """
CALL gds.beta.closeness.stream('longest_paths', {useWassermanFaust: true})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).uniqueId AS name, score AS centrality
ORDER BY centrality DESC;
"""

# improved clonseness
result_data = graph.run(query).to_data_frame()
result_data

,name,centrality
0,russianvsukrainianwar,0.062888
1,24news4u,0.048913
2,puckpootin,0.047259
3,ironmannd88,0.045290
4,hsvusuvavisvajvuz,0.045290
...,...,...
88,treyyingst,0.000000
89,on.dahak,0.000000
90,bbcnews,0.000000
91,visitukraine_,0.000000


In [59]:
query = """
CALL gds.betweenness.stream('longest_paths')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).uniqueId AS user, score
ORDER BY score DESC;
"""
# betweenness
result_data = graph.run(query).to_data_frame()
result_data

,user,score
0,russianvsukrainianwar,50.0
1,24news4u,26.0
2,lukkaventuresyt,16.0
3,world247breakingnews,15.0
4,skynews,11.0
...,...,...
88,fallenangelandpie,0.0
89,diyabutt20,0.0
90,gvc_gary,0.0
91,qtnorthhh,0.0


In [60]:
query = """
CALL gds.pageRank.stream('longest_paths', { maxIterations: 20,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).uniqueId AS page, score
ORDER BY score DESC
LIMIT 10
"""
# pageRank
result_data = graph.run(query).to_data_frame()
result_data

,page,score
0,russianvsukrainianwar,1.044143
1,itsrachelscott,0.857576
2,24news4u,0.824186
3,pray.for.the.world.2022,0.768409
4,bad_vlad_live,0.743258
5,itslavrenova,0.743258
6,skynews,0.727486
7,goldbacked,0.704448
8,world247breakingnews,0.693904
9,now_live,0.678935
